In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, weights=[0.9, 0.1], flip_y=0, random_state=42)
np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

## Experiment 1: Train Logistic Regression Classifier

In [4]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



## Experiment 2: Train Random Forest Classifier

In [5]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       270
           1       0.91      0.70      0.79        30

    accuracy                           0.96       300
   macro avg       0.94      0.85      0.89       300
weighted avg       0.96      0.96      0.96       300



## Experiment 3: Train XGBoost Classifier

In [6]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



## Experiment 4: Handle Class imbalance using SMOTETomek and Train XGBoost Classifier

In [7]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [8]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



## Tracking the experiments using MLflow

In [9]:
models = [
    (
        "Logistic Regression",
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(C=1, solver='liblinear'),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest",
        {"n_estimators":30, "max_depth":3},
        RandomForestClassifier(n_estimators=30, max_depth=3),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder":False, "eval_metric":'logloss'},
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier with SMOTE",
        {"use_label_encoder":False, "eval_metric":'logloss'},
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [10]:
reports = []
for model_name, model_params, model, train_set, test_set in models:
    #print(train_set[1])
    X_train = train_set[0]
    y_train = train_set[1]
    
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**model_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [11]:
reports[0]['macro avg']['f1-score']

0.749791492910759

In [12]:
import mlflow

In [15]:
# Initialize MLFlow
mlflow.set_experiment("Anomaly Detection 2")
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/")

for i, element in enumerate(models):
    model_name = element[0]
    model_params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param("model params", model_params)
        mlflow.log_metric("accuracy", report["accuracy"])
        mlflow.log_metric('recall_class_1', report["1"]["recall"])
        mlflow.log_metric('recall_class_0', report["0"]["recall"])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "xgb-model")
        else:
            mlflow.sklearn.log_model(model, "sklearn-model")
    

2025/10/08 10:32:55 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection 2' does not exist. Creating a new experiment.


2025/10/08 10:32:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/08 10:33:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/837993738209598612/runs/7eb66fae0f134b02a839f927f70f359b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/837993738209598612


2025/10/08 10:33:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/08 10:33:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/837993738209598612/runs/9d501b8212944f209a8de55e24925815
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/837993738209598612


2025/10/08 10:33:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/08 10:33:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/837993738209598612/runs/43896f40bf9647789e16d6743aa945df
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/837993738209598612


2025/10/08 10:33:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/08 10:33:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier with SMOTE at: http://127.0.0.1:5000/#/experiments/837993738209598612/runs/8807ddf2d5b24dc286ce30cb70765997
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/837993738209598612


### Register the Model

In [23]:
model_name = "xgb-model"
run_id = input("Enter the run id: ")
model_uri = f"runs:/{run_id}/{model_name}"
result = mlflow.register_model(
    model_uri, model_name
)

Registered model 'xgb-model' already exists. Creating a new version of this model...
2025/10/08 11:04:34 WARNING mlflow.tracking._model_registry.fluent: Run with id 8807ddf2d5b24dc286ce30cb70765997 has no artifacts at artifact path 'xgb-model', registering model based on models:/m-38c7d641cd9543eab9b20fdb83bdd0dd instead
2025/10/08 11:04:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xgb-model, version 3
Created version '3' of model 'xgb-model'.


### Load the registered model

In [25]:
model_name = "xgb-model"
model_version = 1
# model_uri = f"models:/{model_name}/{model_version}"
model_uri = f"models:/{model_name}@Challenger"

loaded_model = mlflow.xgboost.load_model(model_uri)

y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

### Production Model

In [26]:
from mlflow import MlflowClient

dev_model_uri = f"models:/{model_name}@challenger"
production_model = "anomaly-detection-prod"

client = MlflowClient()
client.copy_model_version(
    src_model_uri = dev_model_uri, dst_name= production_model
)

Successfully registered model 'anomaly-detection-prod'.
Copied version '3' of model 'xgb-model' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1759900763055, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1759900763055, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='8807ddf2d5b24dc286ce30cb70765997', run_link='', source='models:/xgb-model/3', status='READY', status_message=None, tags={}, user_id='', version='1'>

### Prediction through production model

In [27]:
model_uri = f"models:/{production_model}@champion"

loaded_model = mlflow.xgboost.load_model(model_uri)

y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])